# Training an entities recognition model

Importing the required code files

In [1]:
from os import getcwd, path
import sys
import matplotlib.pyplot as plt

BASE_PATH = path.dirname(getcwd())
sys.path.append(BASE_PATH)

from config import START_TAG, STOP_TAG

In [2]:
print(BASE_PATH)

/Users/2359media/Documents/botbot-nlp


The training data must be an array that:
- Contains tuples of (sentence, tags)
- Sentence will be splitted using nltk.wordpunct_tokenize
- Tags will be splitted using .split() - hence spaces by default

Each entity must be separated into 3 kinds of tag: B- (Begin), I- (Inside) and O- (Outside)

_This is to help with separation in the case of consecutive entities_

A `dictionary` to translate from these tags into consecutive indices must be defined
This dictionary will contain:
- The empty token
- `START_TAG` and `END_TAG` tokens (imported from global configs - used internally to indicate start and end of sentence)
- Entities B-, I-, O- tokens

**Sample training data for email recognition:**

In [3]:
training_data = [(
    'My email address is at luungoc2005@gmail.com.',
    '- - - - - - - - - - B-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL -'
), (
    'Contact me at contact@2359media.net.',
    '- - - - - - B-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL -'
), (
    'test.email@microsoft.com is a testing email address',
    'B-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL - - - - - - - - - -'
), (
    'Any inquiries email thesloth_197@gmail.com for assistance',
    '- - - - - - B-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL - - - -'
), (
    'Email addresses include test.noreply@gmail.com hello.vietnam@hallo.org contact@rocket.net',
    '- - - - - - B-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL - B-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL - B-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL'
), (
    'Contact: tester@github.com at any hours',
    '- - - B-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL - - - - - -'
)]

tag_to_ix = {
    '-': 0, # O tag but using '-' for readability
    'B-EMAIL': 1,
    'I-EMAIL': 2,
    START_TAG: 3,
    STOP_TAG: 4
}

In [4]:
from entities_recognition.bilstm.model import SequenceTaggerWrapper
from entities_recognition.bilstm.train import SequenceTaggerLearner
from common.callbacks import PrintLoggerCallback, EarlyStoppingCallback

model = SequenceTaggerWrapper({'tag_to_ix': tag_to_ix})
learner = SequenceTaggerLearner(model)

In [5]:
learner.fit(
    training_data=training_data,
    epochs=50,
    callbacks=[
        PrintLoggerCallback(log_every=5),
        EarlyStoppingCallback()
    ]
)

0m 2s (- 0m 26s) (5 10%) - loss: 12.2995 - accuracy: 0.8070
0m 5s (- 0m 22s) (10 20%) - loss: 5.1858 - accuracy: 0.8809
0m 8s (- 0m 18s) (15 30%) - loss: 2.4357 - accuracy: 0.9804
0m 10s (- 0m 15s) (20 40%) - loss: 1.1892 - accuracy: 0.9804
0m 13s (- 0m 13s) (25 50%) - loss: 0.6972 - accuracy: 1.0000
0m 15s (- 0m 10s) (30 60%) - loss: 0.3502 - accuracy: 1.0000
0m 18s (- 0m 7s) (35 70%) - loss: 0.2324 - accuracy: 1.0000
0m 20s (- 0m 5s) (40 80%) - loss: 0.1559 - accuracy: 1.0000
Best monitor value `loss` == 0.168280 reached. Early stopping


In [6]:
model(['test.email@microsoft.com is a testing email address'])

{'EMAIL': ['test.email@microsoft.com']}

Evaluate model accuracy by using